<a href="https://colab.research.google.com/github/Byskutx/PYTHON_ESILV/blob/main/TPFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ------------------------
# Titre : Analyse des Données DVF
# Auteur : Ethan Binisti, Yasser Bouisk, Bard Bouchabchoub
# Date : 28 janvier 2024
# ------------------------
Ce notebook vise à effectuer une analyse approfondie des données DVF (Demandes de valeurs foncières). Nous allons charger les données, les nettoyer, effectuer une analyse statistique de base, et créer une visualisation. Ensuite, nous intégrerons cette visualisation dans un projet Django pour créer une vitrine dynamique.

##Importation des modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##Importer les données

In [ ]:
url2023 = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20231010-093302/valeursfoncieres-2023.txt"
df = pd.read_csv(url2023, sep="|")

##Affichage des 10 premières lignes du DataFrame pour comprendre la structure des données


In [ ]:
type(df)
df.head(10)

##Vérification du taux des données manquantes dans la data

In [ ]:
df.isnull().sum()/len(df)*100

In [ ]:
df = df.drop(['Identifiant de document', '1 Articles CGI', '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI', 'B/T/Q', 'Prefixe de section', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Identifiant local', 'Nature culture speciale'], axis=1)

Ici, nous constatons que la base brute contient beaucoup de variables qui ont un taux de plus de 90% des données manquantes. Donc, pour ces variables, nous avons exclué dans notre jeu de données.
Pour les autres variables qui ont un taux de moins de 40%, nous les gardons.

##Distributions de fonctionnalités numériques

In [ ]:
df.hist(figsize=(16,16), xrot=-45)
plt.show()

##Quel est le nombre de lignes des données

In [ ]:
df.shape[0]

##Quel est le nombre de colonnes des données

In [ ]:
df.shape[1]

##Quel est le type de données

In [ ]:
df.dtypes

##Vérification des types de données

In [ ]:
df.info()

##Afficher les noms des colonnes du DataFrame restantes

In [ ]:
print(df.columns)

##Classement des types de local les plus vendus avec le prix moyens

In [ ]:
df['Valeur fonciere'] = df['Valeur fonciere'].astype(str)

df['Valeur fonciere'] = pd.to_numeric(df['Valeur fonciere'].str.replace(',', '.'), errors='coerce')
df = df.dropna(subset=['Valeur fonciere'])
resultats = df.groupby('Type local').agg({'Valeur fonciere': ['count', 'mean']})
resultats.columns = ['Nombre de ventes', 'Prix moyen']
resultats['Prix moyen'] = resultats['Prix moyen'].round(0).astype(int)
resultats = resultats.sort_values(by='Nombre de ventes', ascending=False)
top_types_locaux = resultats.head(5)

print("Les cinq types de locaux les plus vendus avec le nombre de ventes et le prix moyen associé :")
print(top_types_locaux)


##Surface moyen d'un local

In [ ]:
df['Surface reelle bati'] = pd.to_numeric(df['Surface reelle bati'], errors='coerce')
surface_moyenne = df['Surface reelle bati'].mean()

print(f"La surface moyenne d'un local est : {surface_moyenne:.2f} m²")


##Nombre de pièces moyennes par local




In [ ]:
df['Nombre pieces principales'] = pd.to_numeric(df['Nombre pieces principales'], errors='coerce')
moyenne_pieces_par_local = df.groupby('Type local')['Nombre pieces principales'].mean()

print("Nombre moyen de pièces principales par type de local :")
print(moyenne_pieces_par_local)


##Répartition de la variable type local

In [ ]:
df['Type local'].value_counts(dropna = False, normalize = True).head()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Type de terrain',fontsize="16")
sns.countplot(y='Type local', data=df)

In [ ]:
plt.figure(figsize=(10,10))
labels = ['Donées Manquantes', 'Maison', 'Appartement', 'Dépendance', 'Local industriel. commercial ou assimilé']
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#fffc52']
area = [40, 16, 14, 26, 0.3]
explode = (0.05,0.05,0.1,0.1,0.1)
plt.pie(area, labels=labels, explode=explode, colors= colors, startangle=65, autopct='%1.1f%%',shadow='True')
plt.title(label= 'Répartition de la variable Type local')
plt.legend()
plt.show()

##Répartition de la variable nature mutation

In [ ]:
df['Nature mutation'].value_counts(dropna = False, normalize = True).head()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Nature Mutation',fontsize="16")
sns.countplot(y='Nature mutation', data=df)

In [ ]:
plt.figure(figsize=(10,10))
labels = ['Vente', 'VEFA', 'Echange', 'Vente Terrain', 'Adjudication']
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#fffc52']
area = [94, 4.1, 1.0, 0.28, 0.89]
explode = (0.05,0.05,0.2,0.3,0.4)
plt.pie(area, labels=labels, explode=explode, colors= colors, startangle=65, autopct='%1.1f%%',shadow='True')
plt.legend()
plt.show()

In [ ]:
import scipy
from scipy.stats import norm
y = np.log(df['Valeur fonciere'])
sns.distplot(y, fit=norm);
fig = plt.figure()

##Classement des 10 codes postals avec le plus de vente de propriétées

In [ ]:
df['Code postal'] = df['Code postal'].astype(str)
df['Valeur fonciere'] = df['Valeur fonciere'].astype(str)
df['Valeur fonciere'] = pd.to_numeric(df['Valeur fonciere'].str.replace(',', '.'), errors='coerce')
df = df.dropna(subset=['Valeur fonciere'])
resultats = df.groupby('Code postal').agg({'Valeur fonciere': ['count', 'mean']})
resultats.columns = ['Nombre de ventes', 'Prix moyen']
resultats['Prix moyen'] = resultats['Prix moyen'].round(0).astype(int)
resultats = resultats.sort_values(by='Nombre de ventes', ascending=False)
top_codes_postaux = resultats.head(10)
print("Les 10 codes postaux avec le plus grand nombre de ventes et le prix moyen associé :")
print(top_codes_postaux)

##Les 10 codes postaux avec le prix moyen le plus haut de France

In [ ]:
df['Code postal'] = df['Code postal'].astype(str)
df['Valeur fonciere'] = df['Valeur fonciere'].astype(str)
df['Valeur fonciere'] = pd.to_numeric(df['Valeur fonciere'].str.replace(',', '.'), errors='coerce')
df = df.dropna(subset=['Valeur fonciere'])
resultats = df.groupby('Code postal').agg({'Valeur fonciere': ['count', 'mean']})
resultats.columns = ['Nombre de ventes', 'Prix moyen']
resultats['Prix moyen'] = resultats['Prix moyen'].round(0).astype(int)
resultats = resultats.sort_values(by='Prix moyen', ascending=False)
top_codes_postaux = resultats.head(10)
print("Les 10 codes postaux avec le prix moyen le plus haut de France et le nombre de ventes associé :")
print(top_codes_postaux)

##Évolution de la Valeur Foncière au fil du temps

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(df['Date mutation'], bins=100, edgecolor='black', alpha=1, label='Date mutation')
plt.xlabel('Date mutation')
plt.ylabel('Valeur fonciere')
plt.title('Évolution de la Valeur Foncière au fil du temps')
plt.legend()
plt.show()

In [ ]:


# Choisissez deux variables parmi celles que vous avez listées
variable_x = 'Surface reelle bati'
variable_y = 'Valeur fonciere'

# Assurez-vous que les colonnes sélectionnées ne contiennent pas de valeurs manquantes
df_cleaned = df[[variable_x, variable_y]].dropna()

# Créez un histogramme bidimensionnel
plt.figure(figsize=(10, 6))
plt.hist2d(df_cleaned[variable_x], df_cleaned[variable_y], bins=(50, 50), cmap='Blues')
plt.colorbar(label='Fréquence')
plt.xlabel(variable_x)
plt.ylabel(variable_y)
plt.title(f'Histogramme bidimensionnel de {variable_x} par rapport à {variable_y}')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df, x='Nombre de lots', y='Valeur fonciere', alpha=0.7, color='skyblue')
plt.title('Nuage de points entre Nombre de lots et Valeur fonciere')
plt.xlabel('Nombre de lots')
plt.ylabel('Valeur fonciere')
plt.show()

In [ ]:
plt.title('Type local par surface terrestre',fontsize="16")
sns.boxplot(y='Type local', x='Surface terrain', data=df)

In [ ]:
url2022 = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20231010-093059/valeursfoncieres-2022.txt"
df_2022 = pd.read_csv(url2022, sep="|")

url2021 = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20231010-092719/valeursfoncieres-2021.txt"
df_2021 = pd.read_csv(url2021, sep="|")

url2020 = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20231010-092355/valeursfoncieres-2020.txt"
df_2020 = pd.read_csv(url2020, sep="|")

url2019 = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20231010-092100/valeursfoncieres-2019.txt"
df_2019 = pd.read_csv(url2019, sep="|")





dfs = [df_2022, df, df_2021, df_2020, df_2019]
df_combined = pd.concat(dfs)
df_combined['Valeur fonciere'] = pd.to_numeric(df_combined['Valeur fonciere'], errors='coerce')
df_combined['Date mutation'] = pd.to_datetime(df_combined['Date mutation'])
pv = pd.pivot_table(df_combined, index=df_combined['Date mutation'].month,
                    columns=df_combined['Date mutation'].year, values='Valeur fonciere', aggfunc='sum')
pv.plot(marker='o', figsize=(10, 6))
plt.title('Land Value by Month for Different Years')
plt.xlabel('Month')
plt.ylabel('Total Land Value')
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()